In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import GroupKFold


In [2]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

In [3]:
lab = pd.read_csv("patients_no_text_sepsis_labels.csv")
text = pd.read_csv("noteevents_preprocessed_text_granular_label.csv")

In [186]:
missing_values_table(lab)

Your selected dataframe has 59 columns.
There are 3 columns that have missing values.


,Missing Values,% of Total Values
o:output_4hourly,1102706,55.4
o:output_total,1102706,55.4
o:PaO2_FiO2,889233,44.6


convert charttime from epoch to hourly for strats

In [180]:
lab["diff"] = lab["m:charttime"].diff()
lab["diff"].loc[lab["step"]==0] = 0
lab["hour"] = lab["diff"]
for i in range(1, len(lab)):
    if lab.loc[i, "hour"] == 0.0:
        continue
    else:
        lab.loc[i, 'hour'] = lab.loc[i-1, 'hour'] + (lab.loc[i, 'diff'] / 3600)




/var/folders/rm/v29286yx225c2ktvpkpwfr5r0000gq/T/ipykernel_21923/62210109.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lab["diff"].loc[lab["step"]==0] = 0


In [182]:
is_16_steps = np.where((lab["hour"]==16) & (lab["step"]==16))
len(np.unique(lab["m:icustayid"].iloc[is_16_steps]))


12683

data in strats format

In [143]:
df = pd.DataFrame({'hour': [1, 2], 'billirubin': [5, 3]}, index=['0', '1'])
df

,hour,billirubin
0,1,5
1,2,3


In [145]:
(df.melt(id_vars=['hour'], var_name='variable',value_name='value', ignore_index=False)
       .sort_values(['hour'])
       .reset_index(drop=True))

,hour,variable,value
0,1,billirubin,5
1,2,billirubin,3


In [ ]:
df

drop meta features, and features that were possibly used for inferring sepsis labels

In [204]:
rows = lab.drop(columns=["m:charttime", "diff", "traj", "step", "r:reward", "a:action", "o:output_4hourly", "o:output_total", "o:PaO2_FiO2", "Unnamed: 0", "Unnamed: 0.1", "m:presumed_onset", "o:SOFA", "o:SIRS", "o:input_4hourly", "o:input_total", "o:GCS", "o:PTT", "o:DiaBP", "o:MeanBP", "o:SpO2", "o:cumulated_balance", "o:Creatinine", "o:Platelets_count", "o:re_admission", "o:mechvent", "o:max_dose_vaso"])
rows

,m:icustayid,o:Arterial_BE,o:Arterial_lactate,o:Arterial_pH,o:BUN,o:Calcium,o:Chloride,o:FiO2_1,o:Glucose,o:HCO3,...,o:Temp_C,o:Total_bili,o:WBC_count,o:Weight_kg,o:age,o:gender,o:paCO2,o:paO2,sepsis_label,hour
0,200003.0,1.071661,-0.560359,1.069870,-0.380498,0.094949,-0.115152,0.270372,-0.431229,1.257526,...,-0.069259,-1.174416,-1.043868,-0.165619,-0.963438,-0.5,-0.374712,-0.689066,1,0.0
1,200003.0,1.071661,-0.560359,1.069870,-0.380498,0.094949,-0.115152,0.270372,-0.431229,1.257526,...,-0.069259,-1.174416,-1.043868,-0.165619,-0.963438,-0.5,-0.374712,-0.689066,1,1.0
2,200003.0,1.071661,-0.560359,1.069870,-0.380498,0.094949,-0.115152,0.270372,-0.431229,1.257526,...,-0.069259,-1.174416,-1.043868,-0.165619,-0.963438,-0.5,-0.374712,-0.689066,1,2.0
3,200003.0,1.071661,-0.560359,1.069870,-0.380498,0.094949,-0.115152,0.270372,-0.431229,1.257526,...,-0.069259,-1.174416,-1.043868,-0.165619,-0.963438,-0.5,-0.374712,-0.689066,1,3.0
4,200003.0,1.071661,-0.560359,1.069870,-0.380498,0.094949,-0.115152,0.270372,-0.431229,1.257526,...,-0.069259,-1.174416,-1.043868,-0.165619,-0.963438,-0.5,-0.374712,-0.689066,1,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991934,299999.0,-0.019540,-0.744383,-0.112773,-0.519833,0.095193,0.191409,-0.340459,-0.353438,-0.175933,...,0.401694,0.181619,0.755655,0.258350,-0.849940,-0.5,-0.076793,-0.483898,0,21.0
1991935,299999.0,-0.019540,-0.744383,-0.112773,-0.519833,0.095193,0.191409,-0.340459,-0.100404,-0.175933,...,0.401694,0.181619,0.755655,0.258350,-0.849940,-0.5,-0.076793,-0.483898,0,22.0
1991936,299999.0,-0.019540,-0.744383,-0.112773,-0.519833,0.095193,0.191409,-0.340459,-0.100404,-0.175933,...,0.401694,0.181619,0.755655,0.258350,-0.849940,-0.5,-0.076793,-0.483898,0,23.0
1991937,299999.0,-0.019540,-0.744383,-0.112773,-0.519833,0.095193,0.191409,-0.340459,-0.100404,-0.175933,...,0.401694,0.181619,0.755655,0.258350,-0.849940,-0.5,-0.076793,-0.483898,0,24.0


In [205]:
strats_data = (rows.melt(id_vars=['hour', 'm:icustayid', 'sepsis_label'], var_name='variable',value_name='value', ignore_index=False)
       .sort_values(['m:icustayid', 'hour'])
       .reset_index(drop=True))



In [206]:
mean_stds = strats_data.groupby('variable').agg({'value':['mean', 'std']})
mean_stds

value         
                            mean      std
variable                                 
o:Arterial_BE       5.458764e-19  1.00000
o:Arterial_lactate  1.483356e-16  1.00000
o:Arterial_pH       4.104030e-14  1.00000
o:BUN              -1.568973e-16  1.00000
o:Calcium          -6.190385e-16  1.00000
o:Chloride         -6.769658e-15  1.00000
o:FiO2_1            1.673816e-16  1.00000
o:Glucose           5.175274e-16  1.00000
o:HCO3              7.725530e-16  1.00000
o:HR               -3.991586e-16  1.00000
o:Hb               -2.797940e-15  1.00000
o:INR              -3.744418e-16  1.00000
o:Magnesium        -7.000505e-17  1.00000
o:PT                3.597625e-16  1.00000
o:Potassium         1.007620e-15  1.00000
o:RR                3.132213e-16  1.00000
o:SGOT              8.740403e-16  1.00000
o:SGPT              2.407745e-16  1.00000
o:Shock_Index       3.377030e-16  1.00000
o:Sodium           -3.812753e-15  1.00000
o:SysBP            -2.779055e-15  1.00000
o:Temp_C           -8.982313e-16  1.00000
o:Total_bili       -2.050873e-17  1.00000
o:WBC_count        -2.295843e-16  1.00000
o:Weight_kg         5.719070e-16  1.00000
o:age              -1.333780e-15  1.00000
o:gender           -6.825335e-02  0.49532
o:paCO2            -2.292463e-15  1.00000
o:paO2              4.173403e-16  1.00000

In [207]:
mean_stds.columns = [col[1] for col in mean_stds.columns]
mean_stds.loc[mean_stds['std']==0, 'std'] = 1

In [208]:
mean_stds

,mean,std
variable,,
o:Arterial_BE,5.458764e-19,1.00000
o:Arterial_lactate,1.483356e-16,1.00000
o:Arterial_pH,4.104030e-14,1.00000
o:BUN,-1.568973e-16,1.00000
o:Calcium,-6.190385e-16,1.00000
o:Chloride,-6.769658e-15,1.00000
o:FiO2_1,1.673816e-16,1.00000
o:Glucose,5.175274e-16,1.00000
o:HCO3,7.725530e-16,1.00000


In [209]:
strats_data = strats_data.merge(mean_stds.reset_index(), on='variable', how='left')

In [222]:
strats_data

,hour,m:icustayid,sepsis_label,variable,value,mean,std
0,0.0,200003.0,1,o:Arterial_BE,1.071661,5.458764e-19,1.00000
1,0.0,200003.0,1,o:Arterial_lactate,-0.560359,1.483356e-16,1.00000
2,0.0,200003.0,1,o:Arterial_pH,1.069870,4.104030e-14,1.00000
3,0.0,200003.0,1,o:BUN,-0.380498,-1.568973e-16,1.00000
4,0.0,200003.0,1,o:Calcium,0.094949,-6.190385e-16,1.00000
...,...,...,...,...,...,...,...
57766226,38.0,299999.0,0,o:Weight_kg,0.258350,5.719070e-16,1.00000
57766227,38.0,299999.0,0,o:age,-0.849940,-1.333780e-15,1.00000
57766228,38.0,299999.0,0,o:gender,-0.500000,-6.825335e-02,0.49532
57766229,38.0,299999.0,0,o:paCO2,-0.705730,-2.292463e-15,1.00000


In [214]:
IDs = pd.read_csv("IDs.csv")
train_IDs = np.unique(IDs["train_ids"][~np.isnan(IDs["train_ids"])])
test_IDs = np.unique(IDs["test_ids"][~np.isnan(IDs["test_ids"])])
val_IDs = np.unique(IDs["val_ids"][~np.isnan(IDs["val_ids"])])
print(np.intersect1d(train_IDs,test_IDs))
print(np.intersect1d(train_IDs,val_IDs))
print(np.intersect1d(test_IDs,val_IDs))

[]
[]
[]


In [223]:
train_data = strats_data.loc[strats_data["m:icustayid"].isin(train_IDs)]
len(train_data)

30549238